In [40]:
import pipeline
import pandas as pd
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
data_cru = pd.read_parquet('heart_2022_no_nans.parquet')

data = pipeline.pipelines(data_cru)
X = data.create()


reportstest = {}
reportstreino = {}

def add_model_report(reports, model_name, y_true, y_pred):
    """ Adiciona o relatório de classificação ao dicionário de relatórios. """
    reports[model_name] = classification_report(y_true, y_pred, output_dict=True)

data_cru = data_cru.drop(columns=['State'])

dados_sem_estados = X.drop(columns=['State'])

#quando for treinar os clusters é necessario que os dados estejam no formato de matriz. Ou seja, nao pode ser um dataframe

dados_em_matriz=dados_sem_estados.values

x = dados_sem_estados.drop('HadHeartAttack', axis=1)
y = dados_sem_estados['HadHeartAttack']

rus = RandomUnderSampler(random_state=42)

dados_sem_estados

,GeneralHealth,PhysicalHealthDays,MentalHealthDays,PhysicalActivities,SleepHours,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,...,AgeCategory_Age 75 to 79,AgeCategory_Age 80 or older,"TetanusLast10Tdap_No, did not receive any tetanus shot in the past 10 years","TetanusLast10Tdap_Yes, received Tdap","TetanusLast10Tdap_Yes, received tetanus shot but not sure what type","TetanusLast10Tdap_Yes, received tetanus shot, but not Tdap",HadDiabetes_No,"HadDiabetes_No, pre-diabetes or borderline diabetes",HadDiabetes_Yes,"HadDiabetes_Yes, but only during pregnancy (female)"
0,0.75,0.133333,0.000000,1.0,0.347826,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.75,0.000000,0.000000,1.0,0.217391,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.75,0.000000,0.000000,0.0,0.304348,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.25,0.166667,0.000000,1.0,0.347826,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.50,0.100000,0.500000,1.0,0.173913,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246017,0.75,0.000000,0.000000,1.0,0.217391,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
246018,0.25,0.000000,0.233333,1.0,0.260870,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
246019,0.50,0.000000,0.500000,1.0,0.260870,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
246020,1.00,0.066667,0.066667,1.0,0.260870,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest , f_classif

def metricas(x,y):
    accuracy = accuracy_score(x, y)
    print(f'Acurácia: {accuracy:.2f}')

    # 2. Precisão
    precision = precision_score(x, y,average='macro')
    print(f'Precisão: {precision:.2f}')

    # 3. Revocação
    recall = recall_score(x, y,average='macro')
    print(f'Revocação: {recall:.2f}')

    # 4. F1 Score
    f1 = f1_score(x, y,average='macro')
    print(f'F1 Score: {f1:.2f}')

    # Exibindo a Matriz de Confusão para referência
    cm = confusion_matrix(x, y)
    print('Matriz de Confusão:')
    print(cm)
    return [accuracy,precision,recall,f1]

x = dados_sem_estados.drop('HadHeartAttack', axis=1)
y = dados_sem_estados['HadHeartAttack']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

x2, y2 = rus.fit_resample(X_train, y_train)


smote = SMOTE(random_state=42)
x3, y3 = smote.fit_resample(X_train, y_train)



selector = SelectKBest(score_func=f_classif, k=10)
X_train_best = selector.fit_transform(x2, y2)
selected_features = x2.columns[selector.get_support()]

xbest_train = x2[selected_features]
xbest_test = X_test[selected_features]

In [42]:
from sklearn.model_selection import GridSearchCV

pipelinerf = Pipeline([
    ('classifier', RandomForestClassifier(random_state=42))
])

param_grid = {
    'classifier__n_estimators': [50, 100, 500,1000],
    'classifier__max_depth': [None, 10, 15,20],
    'classifier__max_features': ['auto', 'sqrt', 'log2'], 
}
 
# Grid Search com validação cruzada
grid_search = GridSearchCV(pipelinerf, param_grid, cv=3, scoring='f1_macro', n_jobs=-1)
grid_search.fit(x3, y3)

# Melhor modelo
best_model = grid_search.best_estimator_


c:\Users\yagod\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning:


48 fits failed out of a total of 144.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\yagod\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\yagod\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\yagod\AppDa

In [43]:
y_pred = best_model.predict(X_test)

y_pred_treino = best_model.predict(x3)

add_model_report(reportstreino,'random forest',y3, y_pred_treino)


add_model_report(reportstest,'random forest',y_test, y_pred)



In [44]:
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],  # Tipo de penalização
    'C': [0.001, 0.01, 0.1, 1, 10, 100],            # Força da penalização (inverso da regularização)
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga'],  # Algoritmo de otimização
    'max_iter': [100, 200, 300],                    # Número máximo de iterações
    'l1_ratio': [0.1, 0.5, 0.7, 1.0] 
}

# Criar o objeto de Regressão Logística
log_reg = LogisticRegression(random_state=42)

# Criar o objeto GridSearchCV
grid_search = GridSearchCV(estimator=log_reg,
                           param_grid=param_grid,
                           cv=3,                # Número de folds para validação cruzada
                           scoring='f1_macro',  # Métrica para avaliação do desempenho
                           n_jobs=-1)           # Número de jobs paralelos

# Ajustar o GridSearchCV aos dados
grid_search.fit(X_train, y_train)

# Melhor modelo encontrado
best_model = grid_search.best_estimator_

c:\Users\yagod\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning:


1080 fits failed out of a total of 2592.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
216 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\yagod\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\yagod\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\yagod\

In [45]:
y_pred = best_model.predict(X_test)

add_model_report(reportstest,'regressao logistica',y_test, y_pred)

y_pred_treino = best_model.predict(X_train)

add_model_report(reportstreino,'regressao logistica',y_train, y_pred_treino)


In [46]:
from matplotlib.pylab import set_state


param_grid = {
    'n_neighbors': [ 7, 9,11,15,19,25],           # Número de vizinhos a considerar
    'weights': ['uniform', 'distance'],    # Função de pesos
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algoritmo de busca
    'p': [1, 2]
}

# Criar o objeto de K-Nearest Neighbors
knn = KNeighborsClassifier()

# Criar o objeto GridSearchCV com f1_score como métrica
grid_search = GridSearchCV(estimator=knn,
                           param_grid=param_grid,
                           cv=3,                # Número de folds para validação cruzada
                           scoring='f1_macro',  # Métrica para avaliação do desempenho
                           n_jobs=-1)           # Número de jobs paralelos

# Ajustar o GridSearchCV aos dados
grid_search.fit(x2, y2)

# Melhor modelo encontrado
best_model = grid_search.best_estimator_


In [47]:
y_pred = best_model.predict(X_test)

add_model_report(reportstest,'knn',y_test, y_pred)


y_pred_treino = best_model.predict(x2)

add_model_report(reportstreino,'knn',y2, y_pred_treino)

In [49]:
param_grid = {
    'depth': [6, 8, 10],                     # Profundidade das árvores
    'learning_rate': [0.01, 0.1, 0.2],      # Taxa de aprendizado
    'n_estimators': [50, 200, 300],         # Número de árvores
}

# Criar o objeto CatBoostClassifier
catboost = CatBoostClassifier(cat_features=[], # Se não houver variáveis categóricas, deixe vazio
                               verbose=0,random_state=42,task_type='GPU', devices='0')     # Desativa o log verbose durante a execução

# Criar o objeto GridSearchCV com f1_score como métrica
grid_search = GridSearchCV(estimator=catboost,
                           param_grid=param_grid,
                           cv=3,                # Número de folds para validação cruzada
                           scoring='f1_macro'  # Métrica para avaliação do desempenho
                          )           # Número de jobs paralelos

# Ajustar o GridSearchCV aos dados
grid_search.fit(x3, y3)

# Melhor modelo encontrado
best_model = grid_search.best_estimator_

In [50]:
y_pred = best_model.predict(X_test)

add_model_report(reportstest,'catboost',y_test, y_pred)


y_pred_treino = best_model.predict(x3)

add_model_report(reportstreino,'catboost',y3, y_pred_treino)

In [51]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l1

seed = 42
tf.random.set_seed(seed)

modelo = Sequential([

Dense(64, input_shape=(X_train.shape[1],), activation='relu', kernel_regularizer=l1(0.01)),
Dropout(0.5),
Dense(32, activation='relu', kernel_regularizer=l1(0.01)),
Dropout(0.5),
Dense(1, activation='sigmoid'),
])
from tensorflow.keras import backend as K

# Definindo a função F1-Score
def f1_scorez(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    f1 = 2 * ((precision * recall) / (precision + recall + K.epsilon()))
    return f1

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',  # Métrica a ser monitorada
    patience=10,         # Número de épocas sem melhora antes de parar o treinamento
    restore_best_weights=True  # Restaurar os pesos do modelo com melhor performance no conjunto de validação
)

modelo.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy',f1_scorez])

history = modelo.fit(x2, y2, epochs=200, validation_split=0.2, batch_size=1024, callbacks=[early_stopping],verbose=0)



y_pred = (modelo.predict(X_test) >0.5).astype("int32")

add_model_report(reportstest,'redes neurais',y_test, y_pred)

y_pred_treino = best_model.predict(x2)

add_model_report(reportstreino,'redes neurais',y2, y_pred_treino)

c:\Users\yagod\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



2307/2307 ━━━━━━━━━━━━━━━━━━━━ 2s 832us/step


In [52]:
import pandas as pd
import plotly.graph_objects as go

def generate_comparison_table(reports):
    """ Gera uma tabela comparativa dos relatórios de classificação dos modelos. """
    data = []

    for model_name, report in reportstreino.items():
        # Adiciona a acurácia
        accuracy = round(report.get('accuracy', 0), 2)
        data.append({'Model': model_name,
                     'Class': 'Accuracy',
                     'Precision': accuracy,
                     'Recall': accuracy,
                     'F1-Score': accuracy,
                     'Support': ''})

        # Adiciona uma linha para cada classe
        for cls, metrics in report.items():
            if cls == 'accuracy':
                continue  # Ignora a linha de 'accuracy'
            row = {'Model': model_name,
                   'Class': cls,
                   'Precision': round(metrics.get('precision', 0), 2),
                   'Recall': round(metrics.get('recall', 0), 2),
                   'F1-Score': round(metrics.get('f1-score', 0), 2),
                   'Support': round(metrics.get('support', 0), 2)}
            data.append(row)

    df_all = pd.DataFrame(data)
    return df_all

# Gerar a tabela comparativa
df_comparison = generate_comparison_table(reportstreino)

# Exibir a tabela com Plotly
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_comparison.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_comparison[col] for col in df_comparison.columns],
               fill_color='lavender',
               align='left'))
])

fig.update_layout(title="Comparação dos relatorios dos treino dos melhores modelos",height=800)
fig.show()


In [53]:
import pandas as pd
import plotly.graph_objects as go


def generate_comparison_table(reports):
    """ Gera uma tabela comparativa dos relatórios de classificação dos modelos. """
    data = []

    for model_name, report in reportstest.items():
        # Adiciona a acurácia
        accuracy = round(report.get('accuracy', 0), 2)
        data.append({'Model': model_name,
                     'Class': 'Accuracy',
                     'Precision': accuracy,
                     'Recall': accuracy,
                     'F1-Score': accuracy,
                     'Support': ''})

        # Adiciona uma linha para cada classe
        for cls, metrics in report.items():
            if cls == 'accuracy':
                continue  # Ignora a linha de 'accuracy'
            row = {'Model': model_name,
                   'Class': cls,
                   'Precision': round(metrics.get('precision', 0), 2),
                   'Recall': round(metrics.get('recall', 0), 2),
                   'F1-Score': round(metrics.get('f1-score', 0), 2),
                   'Support': round(metrics.get('support', 0), 2)}
            data.append(row)

    df_all = pd.DataFrame(data)
    return df_all

# Gerar a tabela comparativa
df_comparison = generate_comparison_table(reportstest)

# Exibir a tabela com Plotly
fig = go.Figure( data=[go.Table(
    header=dict(values=list(df_comparison.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_comparison[col] for col in df_comparison.columns],
               fill_color='lavender',
               align='left'))
])

fig.update_layout(title="Comparação dos relatorios dos teste dos melhores modelos",height=800)
fig.show()
